In [2]:
library(sf)
library(tidyverse)
# library(janitor)
# library(magrittr)
# library(lubridate)
library(fs)
# library(glue)
# library(readxl)
# library(terra)
library(furrr)
# library(exactextractr)
# library(DT)
library(tictoc)

basePath <- path('/explore/nobackup/people/mmacande/srlite/chm_model')
outPath <- path('/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm')
dir_create(outPath)

Linking to GEOS 3.11.1, GDAL 3.6.1, PROJ 9.1.0; sf_use_s2() is TRUE

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   1.0.0 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: future



## Fix Jupyter / R / GDAL
Code blocks below are dealing with gdal path issue on adapt. Still needed as of 2023-08-31

Run once

In [3]:
secondPath <- sf_proj_search_paths()[[2]]
sf_proj_search_paths(secondPath)

# sf_proj_search_paths('/explore/nobackup/people/mmacande/proj')
# Uncomment line below to see proj search paths
# dir_ls(sf_proj_search_paths())

[1] "/home/mmacande/.conda/envs/r4-base-20230102/share/proj"

## Setup output folders

In [4]:
# ifsar_dtm <- rast('/explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_dtm_20221222.tif')
# ifsar_dsm <- rast('/explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_dsm_20221222.tif')

# dir_create(path(outPath, 'train_images'))
# dir_create(path(outPath, 'train_chm'))

# dir_create(path(outPath, 'test_images'))
# dir_create(path(outPath, 'test_chm'))

dir_create(path(outPath, 'train_merged_nodtm_npy/images'))
dir_create(path(outPath, 'train_merged_nodtm_npy/labels'))
dir_create(path(outPath, 'test_merged_nodtm_npy/images'))
dir_create(path(outPath, 'test_merged_nodtm_npy/labels'))


In [8]:
chipPaths_lidar_ifsar_train_test <- bind_rows(
    tibble(path = dir_ls(path(outPath, 'train_ifsar_nodtm_npy/labels'), glob='*.npy'), source='train_ifsar_nodtm_labels'),
    tibble(path = dir_ls(path(outPath, 'train_lidar_nodtm_npy/labels'), glob='*.npy'), source='train_lidar_nodtm_labels'),
    tibble(path = dir_ls(path(outPath, 'train_ifsar_nodtm_npy/images'), glob='*.npy'), source='train_ifsar_nodtm_images'),
    tibble(path = dir_ls(path(outPath, 'train_lidar_nodtm_npy/images'), glob='*.npy'), source='train_lidar_nodtm_images'),
    tibble(path = dir_ls(path(outPath, 'test_ifsar_nodtm_npy/labels'),  glob='*.npy'), source='test_ifsar_nodtm_labels'),
    tibble(path = dir_ls(path(outPath, 'test_lidar_nodtm_npy/labels'),  glob='*.npy'), source='test_lidar_nodtm_labels'),
    tibble(path = dir_ls(path(outPath, 'test_ifsar_nodtm_npy/images'),  glob='*.npy'), source='test_ifsar_nodtm_images'),
    tibble(path = dir_ls(path(outPath, 'test_lidar_nodtm_npy/images'),  glob='*.npy'), source='test_lidar_nodtm_images')) %>%
# rowwise() %>%
mutate(new_path = str_replace(path, 'ifsar_nodtm_npy', 'merged_nodtm_npy'),
       new_path = str_replace(new_path, 'lidar_nodtm_npy', 'merged_nodtm_npy'))

In [9]:
chipPaths_lidar_ifsar_train_test %>% group_by(source) %>% slice_head(n=2)
nrow(chipPaths_lidar_ifsar_train_test)

path,source,new_path
<fs::path>,<chr>,<chr>
/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_ifsar_nodtm_npy/images/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00115_v20231006.npy,test_ifsar_nodtm_images,/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_merged_nodtm_npy/images/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00115_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_ifsar_nodtm_npy/images/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00140_v20231006.npy,test_ifsar_nodtm_images,/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_merged_nodtm_npy/images/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00140_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_ifsar_nodtm_npy/labels/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00115_v20231006.npy,test_ifsar_nodtm_labels,/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_merged_nodtm_npy/labels/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00115_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_ifsar_nodtm_npy/labels/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00140_v20231006.npy,test_ifsar_nodtm_labels,/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_merged_nodtm_npy/labels/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00140_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_lidar_nodtm_npy/images/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00010_v20231006.npy,test_lidar_nodtm_images,/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_merged_nodtm_npy/images/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00010_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_lidar_nodtm_npy/images/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00056_v20231006.npy,test_lidar_nodtm_images,/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_merged_nodtm_npy/images/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00056_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_lidar_nodtm_npy/labels/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00010_v20231006.npy,test_lidar_nodtm_labels,/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_merged_nodtm_npy/labels/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00010_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_lidar_nodtm_npy/labels/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00056_v20231006.npy,test_lidar_nodtm_labels,/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/test_merged_nodtm_npy/labels/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00056_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/train_ifsar_nodtm_npy/images/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00002_v20231006.npy,train_ifsar_nodtm_images,/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/train_merged_nodtm_npy/images/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00002_v20231006.npy


[1] 997924

In [10]:
plan(multicore, workers=36)

In [11]:
tic()
future_pwalk(chipPaths_lidar_ifsar_train_test %>% select(path, new_path), link_create)
toc()


2333.726 sec elapsed


# Done old below

In [18]:
pwalk(chipPaths_lidar_ifsar_train %>% select(path, new_path), link_create)

In [21]:
chipPaths_lidar_ifsar_test <- bind_rows(
    tibble(path = dir_ls(path(outPath, 'test_data_ifsar_npy/labels'),  glob='*.npy'), source='test_ifsar_labels'),
    tibble(path = dir_ls(path(outPath, 'test_data_lidar_npy/labels'),  glob='*.npy'), source='test_lidar_labels'),
    tibble(path = dir_ls(path(outPath, 'test_data_ifsar_npy/images'),  glob='*.npy'), source='test_ifsar_images'),
    tibble(path = dir_ls(path(outPath, 'test_data_lidar_npy/images'),  glob='*.npy'), source='test_lidar_images')) %>%
# rowwise() %>%
mutate(new_path = str_replace(path, 'ifsar_npy', 'npy'),
       new_path = str_replace(new_path, 'lidar_npy', 'npy'))

In [22]:
chipPaths_lidar_ifsar_test %>% group_by(source) %>% slice_head(n=2)
nrow(chipPaths_lidar_ifsar_test)

path,source,new_path
<fs::path>,<chr>,<chr>
/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_ifsar_npy/images/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00115_v20231006.npy,test_ifsar_images,/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_npy/images/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00115_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_ifsar_npy/images/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00140_v20231006.npy,test_ifsar_images,/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_npy/images/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00140_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_ifsar_npy/labels/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00115_v20231006.npy,test_ifsar_labels,/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_npy/labels/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00115_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_ifsar_npy/labels/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00140_v20231006.npy,test_ifsar_labels,/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_npy/labels/chip_WV02_20100709_M1BS_1030010006544A00_CHM_AK_IFSAR_CHM_00140_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_lidar_npy/images/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00010_v20231006.npy,test_lidar_images,/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_npy/images/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00010_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_lidar_npy/images/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00056_v20231006.npy,test_lidar_images,/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_npy/images/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00056_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_lidar_npy/labels/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00010_v20231006.npy,test_lidar_labels,/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_npy/labels/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00010_v20231006.npy
/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_lidar_npy/labels/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00056_v20231006.npy,test_lidar_labels,/explore/nobackup/people/mmacande/srlite/chm_model/20231006_chm/test_data_npy/labels/chip_WV02_20100804_M1BS_103001000617C000_CHM_gliht_chm_2014_32607_00056_v20231006.npy


[1] 50242

In [23]:
plan(multicore, workers=36)

In [24]:
tic()
future_pwalk(chipPaths_lidar_ifsar_test %>% select(path, new_path), link_create)
toc()


176.364 sec elapsed
